In [29]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

ws_driver  = CkipWordSegmenter(model="bert-base")
# pos_driver = CkipPosTagger(model="bert-base")
# ner_driver = CkipNerChunker(model="bert-base")


# Use CPU
# ws_driver = CkipWordSegmenter(device=-1)
# Use GPU:0
# ws_driver = CkipWordSegmenter(device=0)

# Input text
text = [
   "見證程式設計的新時代來臨了由奇標科技公司推出的新書AI神助攻程式設計新境界Github Copilot開發Python如虎添翼提示工程問題分解測試案例除錯你將體驗由AI驅動的程式設計開發Python如虎添翼提示工程問題分解測試案例由AI驅動的程式設計革命不再是繁瑣的語法手冊也不用到處copy程式碼Github Copilot不僅學會了人類的程式設計精華還能透過自然語言協助你提高開發效率本書由兩位資訊教育專家共同撰寫提供了學習Python的新方法書中精心設計的韓式設計循環流程引導你透過實戰演練掌握AI程式設計的關鍵技能奇標科技公司現在提供傳人頻道聽眾10本新書作為抽獎現在就點擊下方資訊欄貼文連結到貼文底下留言一起學程式並標註一位朋友將有機會抽中這本好書喔活動將於5月14號星期二23點59分截止快到資訊欄的貼文連結參加抽獎吧",
]

# Run pipeline
ws  = ws_driver(text)
pos = pos_driver(ws)
ner = ner_driver(text)

# Enable sentence segmentation
ws  = ws_driver(text, use_delim=True)
# ner = ner_driver(text, use_delim=True)
# Disable sentence segmentation
# pos = pos_driver(ws, use_delim=False)
# Use new line characters and tabs for sentence segmentation
# pos = pos_driver(ws, delim_set='\n\t')

print(ws)
# for sentence, sentence_ws in zip(text, ws):
#     print(sentence)
#     print(sentence_ws)
#     print()

Tokenization: 100%|█| 1/1 [00:00<
Inference: 100%|█| 1/1 [00:00<00:
Tokenization: 100%|█| 1/1 [00:00<
Inference: 100%|█| 1/1 [00:00<00:
Tokenization: 100%|█| 1/1 [00:00<
Inference: 100%|█| 1/1 [00:00<00:
Tokenization: 100%|█| 1/1 [00:00<
Inference: 100%|█| 1/1 [00:00<00:

[['見證', '程式', '設計', '的', '新', '時代', '來臨', '了', '由', '奇標', '科技', '公司', '推出', '的', '新', '書', 'AI', '神', '助攻', '程式', '設計', '新', '境界', 'Github', ' Copilot', '開發', 'Python', '如虎添翼', '提示', '工程', '問題', '分解', '測試', '案例', '除錯', '你', '將', '體驗', '由', 'AI', '驅動', '的', '程式', '設計', '開發', 'Python', '如虎添翼', '提示', '工程', '問題', '分解', '測試', '案例', '由', 'AI', '驅動', '的', '程式', '設計', '革命', '不再', '是', '繁瑣', '的', '語法', '手冊', '也', '不用', '到處', 'copy', '程式碼', 'Github', ' Copilot', '不僅', '學會', '了', '人類', '的', '程式', '設計', '精華', '還', '能', '透過', '自然', '語言', '協助', '你', '提高', '開發', '效率', '本', '書', '由', '兩', '位', '資訊', '教育', '專家', '共同', '撰寫', '提供', '了', '學習', 'Python', '的', '新', '方法', '書', '中', '精心', '設計', '的', '韓式', '設計', '循環', '流程', '引導', '你', '透過', '實戰', '演練', '掌握', 'AI', '程式', '設計', '的', '關鍵', '技能', '奇標', '科技', '公司', '現在', '提供', '傳人', '頻道', '聽眾', '10', '本', '新', '書', '作為', '抽獎', '現在', '就', '點擊', '下方', '資訊欄', '貼文', '連結', '到', '貼文', '底下', '留言', '一起', '學', '程式', '並', '標註', '一', '位', '朋友', '將', '有', '機會', '抽中', '這', '本',

In [6]:
# python read file
import os

dataset_path = './dataset'

for file in os.listdir(dataset_path):
    if file.endswith('.txt'):
        file_path = os.path.join(dataset_path, file)

        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # print(content)
        print(f'Context of {file}:')

Context of EP227___德國讀電機博士_在職申請甘苦談_海外留學的心境_ft__子源.txt:
Context of EP228___海底抓取機器人_研發與實作的進程_博士的心境培養_ft__子源.txt:
Context of EP223___中台資工系差異_學生接案經驗談_創新與洞察需求_ft__Yuan_Lin.txt:
Context of EP225___哥倫比亞研究生_紐約生活與衝擊_創新開發的學習_ft__Tony.txt:
Context of EP230___工程師紓壓策略_心理健康的警訊_建立興趣與支持網_ft__Peter.txt:
Context of EP220___工程師媒體經營_人脈建立的橋樑_利他與善的循環_ft__this_web_請網這邊走.txt:
Context of EP222___工程師_X_站立喜劇_理工仔的笑話思維_表演與工作的平衡_ft__藍恩.txt:
Context of EP219___機電系轉前端_自學網頁歷程_做中學的分享_ft__this_web_請網這邊走.txt:
Context of EP229___工程師職場壓力_精神健康的覺察_生活韌性的提升_ft__Peter.txt:


In [1]:
from pymilvus import connections, db

# milvus_conn = connections.connect(host="127.0.0.1", port=19530)

database_name = "docs"
# db.using_database("book")
milvus_conn = connections.connect(
    host="127.0.0.1",
    port="19530",
    # db_name="book"
)

if database_name not in db.list_database():
    db.create_database(database_name)

db.using_database(database_name)
db.list_database()


['default', 'docs']

In [4]:
import ollama
import chromadb

documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

client = chromadb.Client()

# 嘗試取得已存在的 collection，如果不存在則創建新的
try:
    collection = client.create_collection(name="docs")
except Exception as e:
    if "Collection docs already exists" in str(e):
        collection = client.get_collection(name="docs")
    else:
        raise e

# 檢查每個文件 ID 是否已存在，如果存在則跳過新增操作
existing_docs = collection.get()
existing_ids = set(existing_docs['ids'])

# store each document in a vector embedding database
for i, d in enumerate(documents):
    if str(i) in existing_ids:
        print(f"ID {i} already exists, skipping.")
        continue

    response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
    embedding = response["embedding"]
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[d]
    )

ID 0 already exists, skipping.
ID 1 already exists, skipping.
ID 2 already exists, skipping.
ID 3 already exists, skipping.
ID 4 already exists, skipping.
ID 5 already exists, skipping.


In [8]:
# an example prompt
prompt = "What animals are llamas related to?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=1
)
data = results['documents'][0][0]

In [9]:
ollama.pull(model="llama2")


Llamas are members of the camelid family, which means they are closely related to two other animals: vicuñas and camels. All three species belong to the same taxonomic family (Camelidae) and share many similarities in their physical characteristics and behaviors. Specifically, llamas are most closely related to vicuñas, with which they share a common ancestor that lived around 20-30 million years ago. Camels are also members of this family, but they are more distantly related to llamas than vicuñas.


In [10]:
# generate a response combining the prompt and data we retrieved in step 2
output = ollama.generate(
  model="llama2",
  prompt=f"Using this data: {data}. Respond to this prompt: {prompt}"
)

print(output['response'])


Llamas are related to several other animals within the camelid family. Specifically, they are most closely related to vicuñas and camels. All three of these animals belong to the family Camelidae and share many similarities in their physical characteristics and behavior. Here are some key similarities between llamas, vicuñas, and camels:

1. Physical appearance: Llamas, vicuñas, and camels all have long necks, legs, and ears, as well as a distinctive hump on their backs. They also have thick, coarse coats that help them survive in hot, dry climates.
2. Diet: These animals are all herbivores and feed primarily on grasses, leaves, and other vegetation.
3. Habitat: Llamas, vicuñas, and camels are all found in the Andean region of South America, where they inhabit high-altitude grasslands and deserts.
4. Social behavior: These animals are social creatures and often live in herds. They communicate with each other through a variety of vocalizations, including grunts, bleats, and whistles.
5